In [1]:
import numpy as np
from IPython.display import clear_output

from tictactoe import TicTacToe

In [2]:
def hashBoard(board):
    result = 0
    for i in range(9):
        if board[i] == -1:
            result += 2 * 3 ** i
        else:
            result += board[i] * 3 ** i
    return int(result)

def choose_action(board, mode='best', epsilon=0.2):
    global q_table
    q_action = q_table[hashBoard(board)]
    ac_idx = np.argwhere(board == 0).flatten()
    q_action = q_action[board == 0]
    if mode == 'best':
        best_idx = np.argmax(q_action)
        return ac_idx[best_idx]
    if mode == 'advn':
        if np.random.random() > epsilon:
            best_idx = np.argmax(q_action)
            return ac_idx[best_idx]
        else:
            n_action = n_table[hashBoard(board)][board == 0]
            min_idx = np.argmin(n_action)
            return ac_idx[min_idx]
    if mode == 'rand':
        if np.random.random() > epsilon:
            best_idx = np.argmax(q_action)
            return ac_idx[best_idx]
        else:
            rand_idx = np.random.choice(ac_idx.shape[0])
            return ac_idx[rand_idx]
    else:
        raise Exception('没有这种方法。')
    
def updateQTable(board, turn, action, win, done, _board, lr=0.01):
    global q_table
    b_num = hashBoard(board * turn)
    n_table[b_num, action] += 1

    _b_num = hashBoard(_board * -turn)
    if done:
        q_table[_b_num, :] = win * -turn
        q_target = win * -turn * gamma
    else:
        q_target = q_table[_b_num, _board == 0].max() * gamma
    q_table[b_num, action] += (-q_target - q_table[b_num, action]) * lr

In [3]:
gamma = 0.9
lr = 0.01
epsilon = 1
epsilon_delay = 0.99999
epsilon_min = 0.2

ttt = TicTacToe()
q_table = np.zeros([3 ** 9, 9])
n_table = np.zeros([3 ** 9, 9])

In [4]:
q_table = np.load('model/tictactoe_qlearning.npz')['q_table']

In [4]:
total_round = 50000
win_round = 0
draw_round = 0
for cur_round in range(total_round):
    board, turn, win, done = ttt.reset(np.random.choice([1, -1], 1)[0])
    while not done:
        if turn == 1:
            action = choose_action(board * turn, mode='advn', epsilon=epsilon)
        elif turn == -1:
            action = choose_action(board * turn, mode='best')
        _board, _turn, win, done = ttt.step(action)
        
        updateQTable(board, turn, action, win, done, _board, lr=lr)
        
        board = _board
        turn = _turn
    
    epsilon = max(epsilon * epsilon_delay, epsilon_min)
    
    if win == 1:
        win_round += 1
    elif win == 0:
        draw_round += 1
    if (cur_round + 1) % 100 == 0:
        print('已完成%d轮，玩家一胜率为%d-%d-%d。' % (cur_round + 1, win_round, draw_round, 100 - win_round - draw_round))
        win_round = 0
        draw_round = 0
        

已完成100轮，玩家一胜率为35-7-58。
已完成200轮，玩家一胜率为31-6-63。
已完成300轮，玩家一胜率为24-9-67。
已完成400轮，玩家一胜率为20-10-70。
已完成500轮，玩家一胜率为17-9-74。
已完成600轮，玩家一胜率为19-10-71。
已完成700轮，玩家一胜率为19-13-68。
已完成800轮，玩家一胜率为16-6-78。
已完成900轮，玩家一胜率为15-12-73。
已完成1000轮，玩家一胜率为12-10-78。
已完成1100轮，玩家一胜率为16-10-74。
已完成1200轮，玩家一胜率为16-10-74。
已完成1300轮，玩家一胜率为6-14-80。
已完成1400轮，玩家一胜率为12-12-76。
已完成1500轮，玩家一胜率为15-14-71。
已完成1600轮，玩家一胜率为12-13-75。
已完成1700轮，玩家一胜率为13-13-74。
已完成1800轮，玩家一胜率为16-10-74。
已完成1900轮，玩家一胜率为11-13-76。
已完成2000轮，玩家一胜率为11-14-75。
已完成2100轮，玩家一胜率为14-14-72。
已完成2200轮，玩家一胜率为9-14-77。
已完成2300轮，玩家一胜率为5-15-80。
已完成2400轮，玩家一胜率为8-13-79。
已完成2500轮，玩家一胜率为8-13-79。
已完成2600轮，玩家一胜率为9-8-83。
已完成2700轮，玩家一胜率为5-16-79。
已完成2800轮，玩家一胜率为6-11-83。
已完成2900轮，玩家一胜率为7-9-84。
已完成3000轮，玩家一胜率为9-13-78。
已完成3100轮，玩家一胜率为7-12-81。
已完成3200轮，玩家一胜率为8-9-83。
已完成3300轮，玩家一胜率为5-11-84。
已完成3400轮，玩家一胜率为6-13-81。
已完成3500轮，玩家一胜率为8-9-83。
已完成3600轮，玩家一胜率为6-12-82。
已完成3700轮，玩家一胜率为4-14-82。
已完成3800轮，玩家一胜率为5-12-83。
已完成3900轮，玩家一胜率为3-8-89。
已完成4000轮，玩家一胜率为7-11-82。
已完成4100轮，玩家一胜率为3-14-83。
已完成4200轮，玩家一胜率为

In [8]:
# np.savez_compressed('model/tictactoe_qlearning.npz', q_table=q_table)

## 最终测试 - 只有随机玩家0胜才成功。

In [6]:
record = [] # 将输给随机玩家的棋谱记录下来。

total_round = 20000
win_round = 0
draw_round = 0

epsilon = 0.9
epsilon_delay = 0.9999
epsilon_min = 0.1

for cur_round in range(total_round):
    board, turn, win, done = ttt.reset(np.random.choice([1, -1], 1)[0])
    action_list = []
    while not done:
        if turn == 1:
            action = choose_action(turn * board, mode='rand', epsilon=epsilon)
        elif turn == -1:
            action = choose_action(turn * board)
        _board, _turn, win, done = ttt.step(action)
        
        board = _board
        turn = _turn
        
        action_list.append(action.item())
        
    epsilon = max(epsilon * epsilon_delay, epsilon_min)
    
    if win == 1:
        win_round += 1
        record.append(action_list)
    elif win == 0:
        draw_round += 1
    print('\r正在进行第%d轮游戏。。' % (cur_round+1), end='', flush=True)
print('随机玩家战绩为%d-%d-%d。' % (win_round, draw_round, total_round-win_round-draw_round))

正在进行第20000轮游戏。。随机玩家战绩为0-13614-6386。


## 玩家试玩

In [3]:
import time

ttt = TicTacToe()
q_table = np.load('model/tictactoe_qlearning.npz')['q_table']
board, turn, win, done = ttt.reset(np.random.choice([1, -1], 1)[0])
while not done:
    if turn == 1:
        action = choose_action(turn * board)
    elif turn == -1:
        clear_output()
        ttt.draw_board()
        time.sleep(0.1)
        action = int(input())
    board, turn, win, done = ttt.step(action)
clear_output()
ttt.draw_board()
if win == 1:
    print('电脑获胜！')
elif win == -1:
    print('玩家获胜！')
else:
    print('平局！')

o|x|o  o
x|x|o
x|o|x
平局！
